In [ ]:
!pip install -U langchain langchain-community langchain-core langchain-groq
# pip install -U langchain langchain-community langchain-core langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.7
    Uninstalling langgraph-1.0.7:
      Successfully uninstalled langgraph-1.0.7
  Attempting uninstall: langchain
    Found existing installation: langchain 1.2.8
    Uninstalling langchain-1.2.8:
      Successfully uninstalled langchain-1.2.8


In [5]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# LLM
llm = ChatGroq(
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])


In [6]:
store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


## Conversational Buffer Memory

- Stores entire conversation history (nothing removed)

- Idea - LLM always receives full chat → accurate but expensive

In [7]:
chain = prompt | llm

chatbot = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

config = {"configurable": {"session_id": "user1"}}

print(chatbot.invoke({"input": "Hi I am Ranjan"}, config=config).content)
print(chatbot.invoke({"input": "I am learning ML"}, config=config).content)
print(chatbot.invoke({"input": "What am I learning?"}, config=config).content)


Hello Ranjan! It's nice to meet you. How can I assist you today? Do you have any questions, need help with something, or just want to chat? I'm here to help!
That's great to hear that you're learning Machine Learning (ML), Ranjan! ML is a fascinating field that has many applications in various industries. What specific areas of ML are you interested in or currently learning about? Are you working on any projects or would you like some recommendations on resources to get started?

Some popular topics in ML include:

1. Supervised and Unsupervised Learning
2. Deep Learning (e.g., Neural Networks, CNNs, RNNs)
3. Natural Language Processing (NLP)
4. Computer Vision
5. Reinforcement Learning

Let me know if you have any questions or need help with a particular concept. I'd be happy to assist you!
It seems like you're learning Machine Learning (ML), but I'd love to know more specifics. Are you:

1. Taking an online course (e.g., Coursera, Udemy, edX)?
2. Working on a project (e.g., image cla

## Window Memory (Last K messages)

- We manually trim history (that’s how LangChain now does window memory)

In [8]:
from langchain_core.chat_history import InMemoryChatMessageHistory

class WindowMemory(InMemoryChatMessageHistory):
    def add_message(self, message):
        super().add_message(message)
        self.messages = self.messages[-4:]  # keep last 2 conversations

store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = WindowMemory()
    return store[session_id]


In [9]:
chain = prompt | llm

chatbot = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

config = {"configurable": {"session_id": "user1"}}

print(chatbot.invoke({"input": "Hi I am Ranjan"}, config=config).content)
print(chatbot.invoke({"input": "I am learning ML"}, config=config).content)
print(chatbot.invoke({"input": "What am I learning?"}, config=config).content)


Hello Ranjan! It's nice to meet you. Is there something I can help you with or would you like to chat? I'm here to assist you with any questions or topics you'd like to discuss. How's your day going so far?
Machine Learning (ML) is a fascinating field that's constantly evolving. What specific areas of ML are you interested in or currently learning about? Are you working on any projects, or is there a particular concept or technique you're trying to grasp?

Some popular topics in ML include:

1. Supervised and Unsupervised Learning
2. Deep Learning (Neural Networks, CNNs, RNNs, etc.)
3. Natural Language Processing (NLP)
4. Computer Vision
5. Reinforcement Learning

Let me know if you have any specific questions or need help with a particular topic. I'd be happy to assist you in your ML journey! 

Also, are you using any specific libraries or frameworks like TensorFlow, PyTorch, Scikit-learn, etc.?
You're learning Machine Learning (ML). Specifically, you mentioned earlier that you're lea

## Summary Memory (Compressed memory)

- Now we summarize history before sending to model.

In [10]:
from langchain_core.runnables import RunnableLambda

def summarize(history):
    if len(history.messages) < 6:
        return history.messages

    summary_prompt = f"Summarize this conversation:\n{history.messages}"
    summary = llm.invoke(summary_prompt)
    history.messages = [summary]
    return history.messages


In [11]:
chain = prompt | llm

chatbot = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

config = {"configurable": {"session_id": "user1"}}

print(chatbot.invoke({"input": "Hi I am Ranjan"}, config=config).content)
print(chatbot.invoke({"input": "I am learning ML"}, config=config).content)
print(chatbot.invoke({"input": "What am I learning?"}, config=config).content)


Hello Ranjan! It's nice to meet you! I'm glad you're learning Machine Learning (ML). How's your learning journey going so far? Are you enjoying the process of discovering new concepts and techniques in ML?

By the way, I'm here to help you with any questions or doubts you may have about ML. Don't hesitate to ask me anything, and I'll do my best to assist you.

What do you like to do in your free time, Ranjan? Are you interested in any other areas of technology or science besides ML?
You're learning Machine Learning (ML). That's a fascinating field with many exciting applications.

To help you learn ML, I can provide you with some general guidance and resources. Here are a few topics to get you started:

1. **Supervised Learning**: This involves training models on labeled data to make predictions on new, unseen data.
2. **Unsupervised Learning**: This involves discovering patterns and relationships in unlabeled data.
3. **Deep Learning**: This involves using neural networks to analyze a